In [1]:
import json
import pandas as pd
import os
import soundfile as sf
from tqdm import tqdm
from glob import glob
from datasets import Dataset, Audio

audio = Audio(sampling_rate = 16000)

In [2]:
import re
import string

def clean(s):
    s = [c for c in s.lower() if c not in string.punctuation]
    return re.sub(r'[ ]+', ' ', ''.join(s)).strip()

In [3]:
with open('filtered-malaymmlu.json') as fopen:
    d = json.load(fopen)

In [4]:
with open('malaymmlu.json') as fopen:
    d1 = json.load(fopen)

In [5]:
filtered = []
for i, row in tqdm(enumerate(d)):
    filename = os.path.join('malaymmlu', f'{i}.mp3')
    if not os.path.exists(filename):
        continue
        
    if clean(d1[i]['pronunciation']).count('tempat kosong') > 3:
        continue
        
    cleaned = clean(row['answer'])
    if len([w for w in cleaned.split() if len(w) == 1]) > 3:
        continue
        
    y, sr = sf.read(filename)
    if (len(y) / sr) >= 25:
        continue
    
    row['audio_filename'] = filename
    filtered.append(row)

20437it [00:33, 612.47it/s]


In [6]:
len(filtered)

14389

In [7]:
import IPython.display as ipd
ipd.Audio(filtered[-1]['audio_filename'])

In [8]:
dataset = Dataset.from_list(filtered)

In [9]:
dataset = dataset.cast_column("audio_filename", audio)
dataset

Dataset({
    features: ['id', 'prompt', 'answer', 'year', 'subject', 'subject_eng', 'category', 'level', 'options', 'num_options', 'key', 'audio_filename'],
    num_rows: 14389
})

In [10]:
dataset.push_to_hub('malaysia-ai/Speech-Instructions-MalayMMLU')

Uploading the dataset shards:   0%|          | 0/2 [00:00<?, ?it/s]

Map:   0%|          | 0/7195 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/72 [00:00<?, ?ba/s]

Map:   0%|          | 0/7194 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/72 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/malaysia-ai/Speech-Instructions-MalayMMLU/commit/af47465430c8908736449b87fd939b36b985d514', commit_message='Upload dataset', commit_description='', oid='af47465430c8908736449b87fd939b36b985d514', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/malaysia-ai/Speech-Instructions-MalayMMLU', endpoint='https://huggingface.co', repo_type='dataset', repo_id='malaysia-ai/Speech-Instructions-MalayMMLU'), pr_revision=None, pr_num=None)